In [1]:
import asyncio
import websockets
import json

from neuron import h
from neuron.units import ms, mV

%run ../../NoteBooks/HocParse.ipynb
# This sets example_data

h.load_file('C:/Users/Research/Neuron-Web-Interface/Data/DeitcherEtAl2017/Morphs_and_trees/1148.hoc')
h.load_file('stdrun.hoc')

Section soma has 21 points!
Section axon has 172 points!
Section dend has 17 points!
	 132 more sections...


1.0

In [2]:
async def message_loop(websocket):
    while True:
        try:
            data = await websocket.recv()
            data = json.loads(data)

            request_type = data["message"]
            print(request_type, "recieved!")

            match request_type:
                case "LOAD_EXAMPLE":
                    await load_example(websocket)

                case "RENDER":
                    if ("data" in data):
                        await render(websocket, data["data"])
                    pass

        except websockets.exceptions.ConnectionClosedOK: 
            continue

        except json.decoder.JSONDecodeError:
            print("Invalid message:", data)
            break
        
        except Exception as e:
            print("Unkown error:", e)
       
async def load_example(socket):
    await socket.send(json.dumps([json.dumps( {"message" : "EXAMPLE_DATA", "segments": example_data } )]))

async def render(socket, data):
    section_lookup = {sec.name(): sec for sec in h.allsec()}
    output_data = []
    print(len(section_lookup), "sections found...")
    for section_name in section_lookup:
        section = section_lookup[section_name]
        section.Ra = data["Biophysics"]["Ra"] # Axial resistance in Ohm * cm
        section.cm = data["Biophysics"]["cm"] # Membrane capacitance in micro Farads / cm^2

        if ("soma" == section_name):
            has_hh = section.has_membrane("hh")         
            if (has_hh == False):
                section.insert("hh")
            
            for seg in section:
                seg.hh.gnabar = data["Biophysics"]["gnabar"] # Sodium conductance in S/cm2
                seg.hh.gkbar = data["Biophysics"]["gkbar"] # Potassium conductance in S/cm2
                seg.hh.gl = data["Biophysics"]["gl"] # Leak conductance in S/cm2
                seg.hh.el = data["Biophysics"]["el"] # Reversal potential in mV

        if ("dend" in section_name or "apic" in section_name):
            has_pas = section.has_membrane("pas")
            if (has_pas == False):
                section.insert("pas")
            
            for seg in section:
                seg.pas.g = data["Biophysics"]["g"] # Passive conductance in S/cm2
                seg.pas.e = data["Biophysics"]["e"] # Leak reversal potential mV 

    for stim_target in data["Simulation"]["Stimulation"]["Targets"]:
        target_seg = section_lookup[stim_target["Segment"]](stim_target["Location"])
        stim = h.IClamp(target_seg)
        stim.delay = stim_target["Delay"]
        stim.dur = stim_target["Duration"]
        stim.amp = stim_target["Amplitude"]

    for record_target in data["Simulation"]["Recording"]["Targets"]:
        target_seg = section_lookup[record_target["Segment"]](record_target["Location"])
        v = h.Vector().record(target_seg._ref_v)
        t = h.Vector().record(h._ref_t)
        
        output_data.append({ "target": record_target["Segment"], "location": record_target["Location"], "voltage": v, "time": t})

    init_voltage = data["Simulation"]["Recording"]["InitialVoltage"]
    sim_span = data["Simulation"]["Recording"]["Span"]
    h.finitialize(init_voltage * mV)
    h.continuerun(sim_span * ms)

    output_data = [{ 
        "target": recording["target"], 
        "location": recording["location"], 
        "voltage": recording["voltage"].to_python(), 
        "time": recording["time"].to_python()} 
        for recording in output_data]
    
    await socket.send(json.dumps([json.dumps( {"message" : "RENDER_DATA", "data": output_data } )]))

async def main():
    async with websockets.serve(message_loop, "", 8001):
        await asyncio.Future()

print("Starting...")
await main()

Starting...
LOAD_EXAMPLE recieved!
RENDER recieved!
135 sections found...
RENDER recieved!
135 sections found...
LOAD_EXAMPLE recieved!
RENDER recieved!
135 sections found...
LOAD_EXAMPLE recieved!
LOAD_EXAMPLE recieved!
LOAD_EXAMPLE recieved!
LOAD_EXAMPLE recieved!
